In [2]:
a = 0x4e
b = 0x0f

a * 127 + b

9921

In [ ]:
0 == -100
99 = -90



In [96]:
def scale_f(f_in, from_min, from_max, to_min, to_max):
    from_range = from_max - from_min
    to_range = to_max - to_min
    
    f = f_in - from_min
    ret = (f*to_range)/from_range + to_min
    print(ret)
    
    ret = (f_in*to_range) + to_min * 100
    print(ret)

 
    f = f_in * 10 + 10000 
    print(f)
    
    
    
scale_f(126, 125, 130, 0, 100)
    

20.0
12600
11260


In [129]:
global clock_value, prev_seek, current_seek, step, clock_valid


def inc_clock():
    global clock_value, prev_seek
    clock_value += 1
    print("clock: %d" %(clock_value))
    
def process_seek(seek):
    global clock_value, prev_seek, step, prev_clock, clock_estimation, clock_valid
 
    #rel_seek = seek / 100.0
    print("seek", seek)
    
    if seek != prev_seek + 1:
        print("skip")
        prev_seek = seek
        step=0
        clock_valid=0
        return
    
    if step == 0:
        step = 1
        prev_seek = seek
        prev_clock = clock_value
        print("start measuring clock")
        return
    
    if step == 1:
        step = 1
        print(clock_value , prev_clock)
        clock_estimation = clock_value - prev_clock
        
        prev_clock = clock_value
        prev_seek = seek
        
        if clock_estimation == 0:
            print("bad clock estimation")
            return 
        
        print("new clock estimation: %d" % ( clock_estimation))

        clock_period_per_second = 1

        seconds = seek * clock_estimation / clock_period_per_second
        seconds = int(seconds)
        minutes = seconds // 60
        seconds = seconds % 60
        print("%dm%ds" % (minutes, seconds))

        return
    
clock_valid=0
clock_estimation = 0
clock_value=0
prev_seek =0

###
inc_clock()
inc_clock()
inc_clock()
process_seek(2)
inc_clock()
inc_clock()
process_seek(3)
inc_clock()
inc_clock()

process_seek(4)
inc_clock()
inc_clock()
process_seek(50)
inc_clock()
process_seek(51)
inc_clock()
process_seek(52)



clock: 1
clock: 2
clock: 3
seek 2
skip
clock: 4
clock: 5
seek 3
start measuring clock
clock: 6
clock: 7
seek 4
7 5
new clock estimation: 2
0m8s
clock: 8
clock: 9
seek 50
skip
clock: 10
seek 51
start measuring clock
clock: 11
seek 52
11 10
new clock estimation: 1
0m52s


In [ ]:
process_seek(3)


In [133]:
5*60 + 32

332

In [ ]:
saved_clock_algo="""
oo=na
pp=ja+1
ja=oo

if pp==oo then Goto "seek_ok"
ia=0
exit rules, skip Outgoing Action

Label "seek_ok"
if ia==1 then Goto "step_1"
ia=1
ka=hz
exit rules, skip Outgoing Action

Label "step_1"
rr=hz-ka
ka=hz

if rr==0 then exit rules, skip Outgoing Action
ha=rr

// estimate seconds

qq=ha*oo
qq=qq/3

ma=qq



// CLOCK:
//   current clock value:     hz
//   REFERENCE clock value:   ka
//
//   clock units per SEEK:    ha
//   which step we are:       ia

// SEEK:
//   CURRENT location (HR):   na   (nz = lock)
//   PREVIOUS location:       ja
//   REFERENCE location:      ya

//  OUT:
//    calculated timestamp:   ma

"""

In [6]:
from fractions import Fraction # as orig_Fraction


"""
bome variables:
---------------
  local:
    oo-xx

  global:
    ga-g9, ha-h9, i, j, m, n, 
    ya-y9, za-z9

  internal:
    numerator:   G0
    denominator: H0
  
  external:
    msb:    Y0
    lsb:    Z0
  

ddj-1000 outputs:
-----------------
shift: 
  modifier 1 -> 0..1 (tk) -> 0..1 (midi) -> global varible (bome)
tempo range: (COM SHIFT)
  0..13 (TK) -> 0..13 (midi) -> 2..100 (bome, integer) -> speed (ff_2.2) 
tempo position:
  -1..+1 (TK) -> 0..100 (midi) -> -1..+1 (bome, fractional) -> speed (ff_2.2)
stable bpm: (COM SHIFT)
  100..200 (TK) -> 0..100 (midi) -> 100..200 (bome, fractional) -> bpm (ff_3.1)
actual bpm:
  -> tempo_position * bpm (bome, fractional) -> speed (ff_3.1)

beat phase:
  -0.5..0.5 -> 0..90 (midi, relative) -> 0..360 (bome, incremenatl) -> ff_2.2 
cue:
  0 (off)..127 (on) -> set cue to current position


used CCs channel 1 to 4:
------------------------
  normal:   CC 1 -> bome -> led
  especial: CC 2 -> bome -> (block)
  screens:          bome -> CC 20..23


CCs BOME: (new)
---------
  g1/g5 = numerator / denominator Ch_1
  g2/g6 = numerator / denominator Ch_2

  g, h, i, j, k, l, m, n      y,z
    

 IN seek position: CC 90 -> 99

 IN tempo adjust: CC 64 -> g1/g5 (=100)
   S1  CC_110 -   0..75  (75)
   S2  CC_111 -  75..100 (25)
   S3  CC_112 - 100..110 (10)
   S4  CC_113 - 110..120 (10)
   S5  CC_114 - 120..125 (5)
   S6  CC_115 - 125..130 (5)
   S7  CC_116 - 130..135 (5)
   S8  CC_117 - 135..140 (5)
   S9  CC_118 - 140..145 (5)
   S10 CC_119 - 145..155 (10)
   S11 CC_120 - 155..200 (45)
   
 IN BPM value:    CC 65 -> h1/h5 (=1)
   101 -1..-0.75
   102 -0.75..-0.5
   103 -0.5 -0.25
   104 -0.25 0
   105  0  0.25
   106  0.25 +0.50
   107  0.50 +0.75
   108  0.75 +1.0


 IN tempo range:  CC 66 -> i1
    
 CALC tempo perc:       -> j1/j5 (int) -> k1 (ext)
 CALC new bpm:          -> m1/m5 (int) -> k5 (ext)
    
 -------   
 IN beat phase:   CC 67 -> y1 (prev value) -> y5 (degree NEEDLE)
 IN CUE:          CC 68 -> z1 (cue pressed)-> z5 (degree CUE)

 IN shift:        CC 69 -> ga..gb..gc..gd
 
 IN loop size:    CC 70 -> l1 (external)
 IN move size:    CC 71 -> n1 (external)
                        -> n5 ("boolean is loop")
 
 seek position:   CC 72 
    //  TIMER: clock value: hz
    //  TIMER: clock value: ka
    //  CALC:  clock units / 1 hz: ha
    //  STATE clock step: ia
    //  IN: previous seek value: ja
   
 
 
 

CCs DDJ-1000 SCREENS:
---------------------
  needle position: CC 20: 0..2    + CC 52: 0..0x67
  cue position:    CC 23: (same)  + CC 55: ...
  
  bpm value:       CC 21: 0..0x4E + CC 53: 0..0x0F
    no shift: new_bpm / shift: stable_bpm  
  playing speed:   CC 22: (same)  + CC 54: 
    no shift: playing speed / shift: tempo range 
    
  current minute:  note F#4
    -> loop size
  current second:  note G4 
    -> move size
  time mode:       note G#4  
    -> (elapsed = 0 ; remain = -1)
    
  key value:      note C#5   (0..0x18)
  kay change:     note D5    (1..0x19)
    



CCs DDJ-1000 KEEP:
------------------
  led_normal = [0, 2, 4, 5, 7, 11, 15, 19, 31, 33, 34, 35, 38, 41]
  led_screens = [20,21,22,23]
  led_bome = [51,52,53,54,55,56]



more notes:
-----------

local variables: oo..xx 
global variables: gg, yy, zz

G = exact values, from inputs
Y = derived values


CC3 - tempo range
  0..13 -> same
    --> 2%..25%   (%, integer)  -->  G3
   
CC1 - G1 - tempo adjust
  -1..+1 -> 0..100  (%, fake floating point)
    z1 = sign    (1 == "+"; 2 == "-")
    z2 = value   0..100 (on the half side)

  -->  calculated tempo percentage (%)
    y1 = sign
    y2 = value     

    y3 = 0..20000

    
    
    
  
CC2 - G2 - current bpm 
  100..200 -> 0..100  
    -> 100..200  (bpm, integer)

   --> Y2 = calculated BPM
   
       

how to convert ranges:
----------------------
Input  = 0..127
Output = 20..90

O = int(( I * out_range) / in_range) + offset)
  = I * 70 / 127 + 20
  (to add more precision: use 1E3 before and after)
  
  
Pioneer:
--------
needle position: 
  0..360deg = 14 bit value integer
  MSB - 0x2
  LSB - 0x67
  
bpm value:
  0.0 .. 999.9 bpm
  MSB - 0x4E  = 19_998
  LSB - 0x0F
  
play speed:
  -100.0% .. 100.0%  
  MSB: SAME
  LSB: SAME
  
  

"""


def assert_equal_string(a,b):
    if a != b:
        print("ASSERT Error: %s != %s" % (a,b) )
        raise Exception
    else:
        print("ASSERT OK: %s == %s" % (a,b))
        print("\n")

        
        
    
def assert_equal(aa, bb):
    
    try:
        a = aa.value
    except:
        a = aa
    
    try:
        b = bb.value
    except:
        b = bb
        
    
    a = float(a)
    b = float(b)
    
    if abs(a - b) > 0.0001:
        print("ASSERT Error: %s != %s (%s  %s)" % (a, b, type(aa), type(bb)) )
        #print("Error: %s != %s" % (type(a),type(b)) )
        raise Exception
    else:
        print("ASSERT OK: %s == %s    (%s  %s)" % (a, b, type(aa), type(bb)) )
        print("\n")

class Fraction2(Fraction):
    
    
    def __repr__(self):
        return self.__str__()
    
    
    def __repr__2(self):
        self = self.limit_denominator()

        ret = 'Fraction2(%s, %s    %.3f)' % (
            
            self._numerator, self._denominator, self._numerator / self._denominator)        
        return ret
    
    def __str__(self):
        self = self.limit_denominator()
        
        st = '%s/%s' % (self._numerator, self._denominator)
        st = "%.3f(=%s)" % (float(self), st)
        return st
    
    
    def force_denominator(self, new_denominator):
        self = self.limit_denominator()
        
        #print("%s %s" %(self._numerator, self._denominator))
        self._numerator = (self._numerator * new_denominator) / self._denominator
        
        self._denominator = new_denominator
        
        self._numerator = int(self._numerator)
        self._denominator = int(self._denominator)
        
        #print("%s %s" %(self._numerator, self._denominator))
        #self = self.limit_denominator()
        self = Fraction2(self)
        return self
    
    def to_float(self):
        return float(self)
    
    

def test_Fractions():
    a = Fraction2(1123 / 67).force_denominator(100)
    assert_equal(a, 16.76)
    
test_Fractions()    
    
    

ASSERT OK: 16.76 == 16.76    (<class '__main__.Fraction2'>  <class 'float'>)




In [7]:
class fake_float():
    
    def __init__(self, *, value=0, int_digits=4, frac_digits=3, shift_negative=True):
        self.scale = 10 ** frac_digits
        
        self.all_digits = int_digits + frac_digits
        self.frac_digits = frac_digits
        self.int_digits = int_digits
        
        self.type_repr = "FF%d_%d" % (self.int_digits, self.frac_digits)
        
        self.int_max = 1 * (10 ** self.all_digits)
        self.int_min = - self.int_max 
        if self.int_max > 2 ** 30:
            raise "Too high digits"
        
        self.mid_point = self.int_max/2
        
        self.shift_negative= shift_negative
        if not self.shift_negative:
            self.negative_offset = 0
        else:
            self.negative_offset = self.mid_point
            self.int_max *= 2
            self.int_min = 0

        ### DISABLED!!!!
        self.negative_offset = 0
            
        #if type(value) is float:
        #    self = self.from_float(value)
        #    return No

        #elif isinstance(value, Fraction): # numbers.Rational):
        #    self = self.from_Fraction(value)
        #    return self

        #elif type(value) is int:
        #    self.value = value
        #    return self
        
        #else:
        #    self.value=0
        #    return self
 
        self.value=0
        return None

 
    def assert_valid(self):
        self.value = int(self.value)

        if(self.value >= self.int_max - 1):
            print("error: value: %d is too big " % (self.value))
            
        if(self.value <= self.int_min - 1):
            print("error: value: %d is too small " % (self.value))

        
    def set_value(self, value):
        self.value = value
        return self
        


    def to_float(self):
        self.assert_valid()
        f = (self.value - self.int_max/2) / self.scale 
        f = (self.value ) / self.scale 
        return f

    def to_float_str(self):
        self.assert_valid()
        f = self.to_float()
        st = "%.*f" % (self.frac_digits, f)
        return st
    
    
    def from_midi(self, v):
        #v: 0..127
        pass
    
    def mult(self, other):
        a = self.to_float()
        b = other.to_float()
    
        c = a * b
        self.from_float(c)
        return self
    
    def msb(self):
        return int(self.value/128)
    
    def lsb(self):
        return int(self.value%128)
    
    def bit_14(self):
        return "%x:%x" % (self.msb(), self.lsb())
    
    def send_midi(self):
        pass
    
    def copy(self, new_value=None):
        if new_value is None:
            new_value = self.value
            
        a = fake_float(value=new_value, frac_digits=self.frac_digits, int_digits=self.int_digits)
        return a
    
    def show_range(self):
        a = self.copy(- self.int_max + 1)
        mid = self.copy(0)
        b = self.copy(self.int_max - 1)
        
        st = "value: %s     (min: %s  mid: %s   max: %s" % (
                self, a, mid, b)
        return st
    
    def __repr__(self):
        return "%s(%s(=%s))" % ( self.type_repr, self.to_float_str(), self.value)
        
    def __str__(self):
        return self.__repr__()

    def adjust_negative(self):
        self.value += self.negative_offset
    
    def from_fraction(self, f):
        #print(f)
        #print(float(f))
        f = Fraction2(f)
        f = f.force_denominator(self.scale)
                
        self.value = f._numerator
        
        self.adjust_negative()
        return self

    def from_float(self, f):
        self.value = f * self.scale #+ self.int_max/2
        self.adjust_negative()
        self.assert_valid()
        return self
    
    
class ff_4_3(fake_float):
    def __init__(self, value=0):
        
        int_digits=4
        frac_digits=3
        shift_negative=False
        super().__init__(shift_negative=shift_negative,value=value,int_digits=int_digits, frac_digits=frac_digits)
        return None

class ff_3_0(fake_float):
    def __init__(self, value=0):
        
        int_digits=3
        frac_digits=0
        shift_negative=False
        super().__init__(shift_negative=shift_negative,value=value,int_digits=int_digits, frac_digits=frac_digits)
        return None

    
class ff_2_2(fake_float):
    def __init__(self, value=0):
        
        int_digits=2
        frac_digits=2
        shift_negative=True
        super().__init__(shift_negative=shift_negative,value=value,int_digits=int_digits, frac_digits=frac_digits)
        return None
    
    
class ff_3_1(fake_float):
    def __init__(self, value=0):
        
        int_digits=3
        frac_digits=1
        shift_negative=False
        super().__init__(shift_negative=shift_negative,value=value,int_digits=int_digits, frac_digits=frac_digits)
        return None    
       
def test_fake_float():
    a = ff_3_0().from_float(360).bit_14()
    assert_equal_string(a, '2:68')

    b = ff_3_1().from_fraction(Fraction2(999.9)).bit_14()
    assert_equal_string(b, '4e:f')

    c = ff_2_2().from_float(5).value
    assert_equal(c, 500 )

    d = ff_2_2().from_float(5).bit_14()
    assert_equal_string(d, '3:74')
    
    
test_fake_float()


ASSERT OK: 2:68 == 2:68


ASSERT OK: 4e:f == 4e:f


ASSERT OK: 500.0 == 500.0    (<class 'int'>  <class 'int'>)


ASSERT OK: 3:74 == 3:74




In [8]:
def scale(*args):
    a = scale_f(*args)
    a = int(a)
    return a
    
def scale_f(f_in, from_min, from_max, to_min, to_max):
    from_range = from_max - from_min
    to_range = to_max - to_min
    
    f = f_in - from_min
    ret = (f*to_range)/from_range + to_min
    return ret



def print2(debug, v):
    print(v, type(v))

 



class midi():
    
    def assert_valid(self):
        # test valid MIDI value
        if self.value <0 or self.value>127:
            print("value outside MIDI: %s" % (self.value))
            raise
            
    def __init__(self, value, *, ddj_type, tk_min, tk_max, midi_min, midi_max):
        self.tk_min = tk_min
        self.tk_max = tk_max
        self.midi_min = midi_min
        self.midi_max = midi_max
        
        self.value = value
        self.ddj_type = ddj_type
        
        self.debug=False
        
        return None
        
    def from_fraction(self, value):
        self.from_float(value)
        return self
        
    def from_float(self, value):
        try:
            value = value.value
        except:
            pass
        
        self.value = Fraction2(value).limit_denominator()
        
        return self
        
    def to_float(self):
        return float(self.value)
    
        
    def tk_send(self, f=None):
        if not f is None:
            self.value=f
            
        ret = scale(self.value, self.tk_min, self.tk_max, self.midi_min, self.midi_max)
        self.value = ret
        return self

    def bome_recv_float(self, f=None):
        if not f is None:
            self.value=f
        
        ret = scale(self.value, self.midi_min, self.midi_max, self.tk_min, self.tk_max)
        self.value = ret
        return self

    def do_send_and_receive(self):
        self.tk_send()
        self = self.bome_recv()
        return self


    
    def __repr__(self):
        return "midi(%s   %.3f      TK: %s..%s   MIDI: %s..%s   DDJ_out: %s)" % (
            self.value, float(self.value),
            self.tk_min, self.tk_max,
            self.midi_min, self.midi_max,
            self.ddj_type,
        )
            
    def __str__(self):
        return self.__repr__()
        
    def as_fake(self):
        return fake_float(self.value)
    
    @property
    def get_value(self):
        return self.value    
    
    def print(self, v):
        if self.debug:
            print(v, type(v))
    
    
    
    def bome_recv(self, f=None):
        # out: fakefloat
        self.assert_valid()

        midi_max = Fraction2(self.midi_max)
        
        #return midi_max
        midi_min = Fraction2(self.midi_min)
        tk_max = Fraction2(self.tk_max)
        tk_min = Fraction2(self.tk_min)
        
        midi_range = midi_max - midi_min
        tk_range = tk_max - tk_min
        
        
        self.print("midi: %s %s range: %s" % (midi_min, midi_max, midi_range))
        self.print("TK: %s %s range: %s" % (tk_min, tk_max, tk_range))
 
        #######
        # f = f - a_min
        # ret = (f*b_range)/a_range + b_min

        v = Fraction2(int(self.value),1)
        self.print(v)
        
        v = v - midi_min
        self.print(v)

        v = v * tk_range
        self.print(v)

        v = v / midi_range
        self.print(v)

        
        v = v + tk_min
        self.print(v)
            
        self.value = Fraction2( v)    
        return self
    

        
    def __mul__(self, other):
        v = self.value * other.value
        self.value = v
        return self

    def bome_send(self):
        f1 = self.value
        f2 = self.ddj_type()
        f2 = f2.from_fraction(f1)
        
        return f2
        
    def do_debug(self):
        self.debug = True
        return self
        
    def no_debug(self):
        self.debug = False
        return self
        
        
class midi_bpm(midi):
    def __init__(self, value=0):
        tk_min=100.0
        tk_max=200.0
        midi_min=0
        midi_max=100

        ddj_type = ff_3_1

        super().__init__(value=value, ddj_type=ddj_type,
                         tk_min=tk_min, tk_max=tk_max, midi_min=midi_min, midi_max=midi_max)
        return None

    
class midi_tempo_adjust(midi):
    def __init__(self, value=0):
        
        tk_min=-1.0
        tk_max=1.0
        midi_min=0
        midi_max=100
        
        ddj_type = ff_2_2

        super().__init__(value=value, ddj_type=ddj_type,
                         tk_min=tk_min, tk_max=tk_max, midi_min=midi_min, midi_max=midi_max)
        return None


    
def calc_requested_tempo(tempo_range, adj):
    """
    tempo in percentage. eg, 5%
    """
    print("TEMPO RANGE:%s  ADJ: %s" % ( tempo_range, adj))
    
    
    try:
        adj = adj.value
    except:
        pass
    
    
    v = tempo_range * adj
    #v = 1 + (v / 100)
    print("V: %s" % (v))
    ret =  v

    ret = Fraction2(ret)
    
    ret = midi_tempo_adjust().from_fraction(ret)
    return ret
    

def calc_new_bpm(bpm, tempo_range, adj):
    """
    calcs new bpm. inputs are the original ones
    """
    
    print("BPM: %s  TEMPO RANGE:%s  ADJ: %s" % (bpm, tempo_range, adj))
    
    try:
        bpm = bpm.value
    except:
        pass
    
    try:
        adj = adj.value
    except:
        pass
    
        
    tempo_range = tempo_range
    
    v = tempo_range * adj
    v = v/100
    v = v + 1
    ret = v * bpm

    ret = Fraction2(ret)
    ret = midi_bpm().from_fraction(ret)
    
    return ret    
    
    
def test_calc_bpm():
    assert_equal(calc_new_bpm(100,10,1),   110.0)
    assert_equal(calc_new_bpm(100,10,0.5), 105.0)
    assert_equal(calc_new_bpm(100,10,0),   100.0)
    assert_equal(calc_new_bpm(120,5,1),   120+6)
    assert_equal(calc_new_bpm(120,5,0.5),   120+3)

    
def only_do_calcs(tempo_range_, bpm_, adj_):
    global tempo_range, bpm, adj,     new_bpm, new_perc
    
    tempo_range = tempo_range_
    bpm = bpm_
    adj = adj_
    
    bpm = midi_bpm().from_float(bpm).tk_send().bome_recv()
    adj = midi_tempo_adjust().from_float(adj).tk_send().bome_recv()

    new_perc = calc_requested_tempo(tempo_range, adj)
    new_bpm = calc_new_bpm(bpm, tempo_range, adj)
    
    return ( tempo_range, bpm, adj, new_bpm, new_perc)


def test_bome_calcs():
    global tempo_range, bpm, adj, new_bpm, new_perc
    
    
    tempo_range = 10

    print("Testing receiving")
    bpm = midi_bpm().from_float(133.3).tk_send().bome_recv()
    assert_equal(bpm, 133.0)

    adj = midi_tempo_adjust().from_float("-0.45").tk_send().bome_recv()
    assert_equal(adj, -0.460)


    print("Testing calculations")
    new_perc = calc_requested_tempo(tempo_range, adj)
    assert_equal(new_perc, -4.6)
    
    new_bpm = calc_new_bpm(bpm, tempo_range, adj)
    assert_equal(new_bpm, 126.882)

    
    print("Testing sending out")
    send_perc = new_perc.bome_send()
    assert_equal(send_perc, -460)

    send_bpm = new_bpm.bome_send()
    assert_equal(send_bpm, 1268)

    
   
#test_calc_bpm()
    
test_bome_calcs()

#only_do_calcs(10, 133.3, -0.45, )


Testing receiving
ASSERT OK: 133.0 == 133.0    (<class '__main__.midi_bpm'>  <class 'float'>)


ASSERT OK: -0.46 == -0.46    (<class '__main__.midi_tempo_adjust'>  <class 'float'>)


Testing calculations
TEMPO RANGE:10  ADJ: midi(-0.460(=-23/50)   -0.460      TK: -1.0..1.0   MIDI: 0..100   DDJ_out: <class '__main__.ff_2_2'>)
V: -23/5
ASSERT OK: -4.6 == -4.6    (<class '__main__.midi_tempo_adjust'>  <class 'float'>)


BPM: midi(133.000(=133/1)   133.000      TK: 100.0..200.0   MIDI: 0..100   DDJ_out: <class '__main__.ff_3_1'>)  TEMPO RANGE:10  ADJ: midi(-0.460(=-23/50)   -0.460      TK: -1.0..1.0   MIDI: 0..100   DDJ_out: <class '__main__.ff_2_2'>)
ASSERT OK: 126.882 == 126.882    (<class '__main__.midi_bpm'>  <class 'float'>)


Testing sending out
ASSERT OK: -460.0 == -460.0    (<class '__main__.ff_2_2'>  <class 'int'>)


ASSERT OK: 1268.0 == 1268.0    (<class '__main__.ff_3_1'>  <class 'int'>)




In [14]:
global tempo_range, bpm, adj, new_bpm, new_perc

tempo_range = 5


def assert_equal_manual(ret, n, d):
    n = int(n)
    d = int(d)
    
    try:
        tmp = Fraction2(n,d)
    except:
        print("error fraction3: %s %s" % (n,d) )
        raise
    
    print("")
    print("assert_manual: expected: %s ; tmp_fraction = %s ; n = %s d = %s" % (ret,  tmp, n, d))
    assert_equal(ret, tmp)
    

###########

def manual_recv_bpm(bpm):
    from_midi = midi_bpm().from_float(bpm).tk_send()
    
    from_midi_value = from_midi.value
    correct_outcome = from_midi.bome_recv().value
    print("from midi value: %s   outcome: %s" % (from_midi_value, correct_outcome))

    ## BOME CODE STARTS HERE
    ret = from_midi_value + 100
    
    assert_equal(ret, correct_outcome)


def manual_recv_adj(adj):
    from_midi = midi_tempo_adjust().from_float(adj).tk_send()
    
    from_midi_value = from_midi.value
    correct_internal = from_midi.bome_recv().value
    print("from midi value: %s   outcome: %s" % (from_midi_value, correct_internal))

    ## BOME CODE STARTS HERE
    ret_n = from_midi_value - 50
    ret_n = ret_n * 2
    ret_d = 100
    ### BOME CODE ENDS HERE
    
    assert_equal_manual(correct_internal, ret_n, ret_d)
    
    
def manual_calc_requested_tempo(tempo_range, adj):
    from_midi = midi_tempo_adjust().from_float(adj).tk_send()
    from_midi_value = from_midi.value
    correct_internal = from_midi.bome_recv().value

    correct_internal = calc_requested_tempo(tempo_range, correct_internal)
    print("from midi value: %s   outcome: %s" % (from_midi_value, correct_internal))

    ## BOME CODE STARTS HERE
    ret_n = from_midi_value - 50
    ret_n = ret_n * 2
    ret_d = 100    
    ret_n = ret_n * tempo_range
    ### BOME CODE ENDS HERE

    assert_equal_manual(correct_internal, ret_n, ret_d)

    
def manual_send_requested_tempo(tempo_range, adj):
    from_midi = midi_tempo_adjust().from_float(adj).tk_send()
    from_midi_value = from_midi.value
    correct_internal = from_midi.bome_recv().value
    
    correct_internal = calc_requested_tempo(tempo_range, correct_internal)
    correct_external = correct_internal.bome_send().value
    print("from midi value: %s  correct outcome: %s" % (from_midi_value, correct_external))

    ## BOME CODE STARTS HERE
    ret_n = from_midi_value - 50
    ret_n = ret_n * 2
    ret_d = 100
    
    ret_n = ret_n * tempo_range
    ret_d = ret_d * 1

    ret_n2 = ret_n * 100
    ret_n2 = ret_n2 / ret_d
    
    ret_d = 100
    ret_d = ret_d / 100
    ### BOME CODE ENDS HERE
    
    assert_equal_manual(correct_external, ret_n, ret_d)

    
def manual_calc_and_send_requested_bpm_stable(tempo_range, adj, bpm):
    grand_answer = calc_new_bpm(bpm, tempo_range, adj,)
    print("GRAND ANSWER:  tempo_range: %s  adj: %s  bpm: %s --> %s\n\n" % (tempo_range, adj, bpm, grand_answer))

    #return
    
    ### ADJ
    adj_from_midi = midi_tempo_adjust().from_float(adj).tk_send()
    adj_from_midi_value = adj_from_midi.value
    adj_internal = adj_from_midi.bome_recv().value
    
    print("ADJ:  from midi value: %s  internal: %s   \n" % (
            adj_from_midi_value, adj_internal))

    ### BPM
    bpm_from_midi = midi_bpm().from_float(bpm).tk_send()
    bpm_from_midi_value = bpm_from_midi.value
    bpm_internal = bpm_from_midi.bome_recv().value
    
    print("BPM:  from midi value: %s  internal: %s   \n" % (
            bpm_from_midi_value, bpm_internal))
       
    newbpm_internal = calc_new_bpm(bpm_internal, tempo_range, adj_internal)
    newbpm_external = newbpm_internal.bome_send().value
    print("NEW_BPM:    internal: %s   external: %s\n" % (
            newbpm_internal, newbpm_external))
    
    #return
    
    ## BOME CODE STARTS HERE
    print("BOME CODE")
    
    ## INIT
    g1 = 0
    h1 = 0
    
    ## get inputs
    pp1 = adj_from_midi_value
    pp2 = bpm_from_midi_value
    g3 = tempo_range
        
    ## IN: bpm
    g2 = pp2 + 100
    h2 = 1
    
    ## IN: adj
    g1 = pp1 - 50
    g1 = g1 * 2
    h1 = 100



    # CALC: tempo perc
    g4 = g1 * g3
    h4 = h1 * 1

    print("step1: tempo adjust in %", g4, h4)

    g5 = g4 
    h5 = h4 * 100 

    g5 = g4 + h5
    h5 = h5

    g5 = g5 * g2
    h5 = h5 * h2
    
    print("final internal: ", g5,h5)
    
    ## emulate send
    ss = h5 / 10
    oo = g5 / ss
    
    print("final oo: ", oo)
    
    assert_equal_manual(newbpm_external, oo, 1)
    
    y5 = oo // 127
    z5 = oo % 127
    print("final bit_14: ", y5,z5)
    ### BOME CODE ENDS HERE    
    
    
    
    
    
def manual_calc_and_send_requested_bpm(tempo_range, adj, bpm):
    #grand_answer = calc_new_bpm(bpm, tempo_range, adj,)
    #print("GRAND ANSWER:  tempo_range: %s  adj: %s  bpm: %s --> %s\n\n" % (tempo_range, adj, bpm, grand_answer))

    ### ADJ
    adj_from_midi = midi_tempo_adjust().from_float(adj).tk_send()
    adj_from_midi_value = adj_from_midi.value
    adj_internal = adj_from_midi.bome_recv().value
    
    print("ADJ:  from midi value: %s  internal: %s   \n" % (
            adj_from_midi_value, adj_internal))
    
    ### BPM
    bpm_from_midi = midi_bpm().from_float(bpm).tk_send()
    bpm_from_midi_value = bpm_from_midi.value
    bpm_internal = bpm_from_midi.bome_recv().value
    
    print("BPM:  from midi value: %s  internal: %s   \n" % (
            bpm_from_midi_value, bpm_internal))
       
    newbpm_internal = calc_new_bpm(bpm_internal, tempo_range, adj_internal)
    newbpm_external = newbpm_internal.bome_send().value
    print("NEW_BPM:    internal: %s   external: %s\n" % (
            newbpm_internal, newbpm_external))
    
    #return
    
    ## BOME CODE STARTS HERE
    print("BOME CODE")
    
    ## INIT
    g1 = 0
    g5 = 1
    
    h1 = 120
    h5 = 1
    
    i1 = 10
    i5 = 1
    
    
    
    
    ## get inputs
    pp = adj_from_midi_value
    oo = bpm_from_midi_value
    i1 = tempo_range
        
    
    ## IN: adj
    g1 = pp - 50
    g1 = g1 * 2
    g5 = 100

    ## IN: bpm
    h1 = oo + 100
    h5 = 1

    # CALC: tempo perc
    j1 = g1 * i1
    j5 = g5 * 1

    print("step1: tempo adjust in percentage: %s %s" % ( j1, j5))
    ss = j5 / 10
    oo = j1 / ss
    
    k1 = oo 
    
    
    print("step1: EXTERNAL tempo adjust  %s %s" % ( k1, k5))
    print("")
    ######
    
    ## CALC: new BPM
    # 2)
    m1 = j1 
    m5 = j5 * 100 

    # 3)
    m1 = m1 + m5
    m5 = m5

    # 4)
    m1 = m1 * h1
    m5 = m5 * h5
    
    print("step2: internal new bpm: ", m1, m5)
    
    
    """
    algorithm:
    1) v = tempo_range * adj
    2) v = v/100
    3) v = v + 1
    4) ret = v * bpm
    """
    
    ## emulate send
    ss = m5 / 10
    oo = m1 / ss
    
    print("final oo: ", oo)
    
    assert_equal_manual(newbpm_external, oo, 1)
    
    n1 = oo 
    
    print("step2: EXTERNAL new bpm: ", n1, n5)
    ### BOME CODE ENDS HERE
    
    
    
    
def test_manual_operations():
    manual_recv_bpm(121.2)
    manual_recv_adj(0.5)
    manual_calc_requested_tempo(5, 0.5)
    manual_send_requested_tempo(5, 0.5)
    manual_calc_and_send_requested_bpm(10, 0.5, 121)

#test_manual_operations()

manual_calc_and_send_requested_bpm(10, 0.5, 121)



ADJ:  from midi value: 75  internal: 0.500(=1/2)   

BPM:  from midi value: 21  internal: 121.000(=121/1)   

BPM: 121.000(=121/1)  TEMPO RANGE:10  ADJ: 0.500(=1/2)
V: 21/20
NEW_BPM:    internal: midi(2541/20   127.050      TK: 100.0..200.0   MIDI: 0..100   DDJ_out: <class '__main__.ff_3_1'>)   external: 1270

BOME CODE
step1: tempo adjust in percentage: 500 100
step1: EXTERNAL tempo adjust  0 50

step2: internal new bpm:  1270500 10000
final oo:  1270.5

assert_manual: expected: 1270 ; tmp_fraction = 1270.000(=1270/1) ; n = 1270 d = 1
ASSERT OK: 1270.0 == 1270.0    (<class 'int'>  <class '__main__.Fraction2'>)


step2: EXTERNAL new bpm:  10 0


In [23]:
def emulate_needle():
    
    y1 = 0  #// prev
    z1 =0   #// deg
    oo = 0  #// delta

    for j in range(5):
        for i in range(9):
            ii = i * 10
            #print(i)

            oo = ii - y1

            if oo < -20:
                oo = oo + 90

            z1 = z1 + oo
            z1= z1 % 360
            print(i, z1)

            y1 = ii

emulate_needle()


0 0
1 10
2 20
3 30
4 40
5 50
6 60
7 70
8 80
0 90
1 100
2 110
3 120
4 130
5 140
6 150
7 160
8 170
0 180
1 190
2 200
3 210
4 220
5 230
6 240
7 250
8 260
0 270
1 280
2 290
3 300
4 310
5 320
6 330
7 340
8 350
0 0
1 10
2 20
3 30
4 40
5 50
6 60
7 70
8 80
